In [1]:
import networkx as nx 
import matplotlib.pyplot as plt 
import scipy as sp
import numpy as np
from matplotlib.lines import Line2D
import collections
import random
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import kernighan_lin_bisection
import community as community_louvain
from networkx.algorithms.community import label_propagation_communities
